In [1]:
# 1. 셀레니움 환경 구성

# 라이브러리 선언
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.action_chains import ActionChains    # send_keys가 ActionCahins로 연결이 안 되면 삭제 예정
import bs4
import pandas as pd
import time
sleepTime = 0.5

# 드라이버 위치 설정
# driver_loc = "./chromedriver"
driver_loc = "./chromedriver.exe"

# 드라이버 옵션 설정
options = webdriver.ChromeOptions()
# options.add_argument("window-size=2280x1440") # 파이썬이 크롬창을 띄울 것인데 창 크기 선택
# options.add_argument('headless')                           
# options.add_argument('disable-gpu')
# options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")

# 웹 드라이버 정의
driver = webdriver.Chrome(driver_loc, options=options)

In [2]:
# 2. 스마트밴드 상품 목록 페이지로 이동

# 웹페이지 이동

# 다나와 홈페이지로 이동
danawaUrl = "http://www.danawa.com/"
driver.get(danawaUrl)

# 메인 메뉴 클릭
catMainXpath = '//*[@id="danawa_header"]/div[2]/div/div[1]/a/span[2]'
catMainMenu = driver.find_element_by_xpath(catMainXpath)
catMainMenu.click()
time.sleep(sleepTime)

# 태블릿/모바일 클릭
catSubXpath = '//*[@id="sectionExplodeLayer"]/div[1]/ul/li[3]/a'
catSubMenu = driver.find_element_by_xpath(catSubXpath)
catSubMenu.send_keys(Keys.ENTER)
time.sleep(sleepTime)

# 스마트밴드 클릭 (.clikc()으로 이동시 플래시 광고가 뜨면 광고를 누르기 때문에 .send_keys로 이동)
catTargetXpath = '//*[@id="categoryExplodeLayer12"]/div[2]/div[2]/ul/li[3]/a'
catTargetMenu = driver.find_element_by_xpath(catTargetXpath)
catTargetMenu.send_keys(Keys.ENTER)
time.sleep(sleepTime)

In [3]:
# 3. 데이터 수집

rowList = []
colHeaderList = ['제품명','최저가','사이트주소']

이 아래 부분 코드는 돌아가는데 to_csv 저장시 오류 발생. / ValueError: 81 columns passed, passed data had 80 columns<br>
이 아래 코드 어딘가에서 수량 체크가 잘못됨.
해당 부분 수정사항은 현재 상품명 구하는 코드를 divTag1, divTags2를 줄였음.

In [4]:
for i in range(0, 999):

    try:
        # 전체 상품 웹페이지 이동 설정
        pageSelector = '#productListArea > div.prod_num_nav > div > div > a:nth-child(' + str(i + 1) + ')'
        page = driver.find_element_by_css_selector(pageSelector)
        page.click()
        time.sleep(sleepTime)

    except:
        break

    # 전체 상품 웹페이지 소스코드 변수에 저장
    smartBandHtml = driver.page_source

    # 태그 정보만 추출하여 변수에 저장
    bs = bs4.BeautifulSoup(smartBandHtml, 'html.parser')
    
    # divTag1, divTag2가 있던 부분임. 아래 상품명 구할 때 말고는 더이상 안 쓰는거라면 find, findAll 필요 없어서 줄일 수 있음.
    #productListArea > div.main_prodlist.main_prodlist_list > ul > li > div
    #productItem9301353 > div
    
#     # 상품 리스트가 있는 영역을 구함
#     totalProduct = bs.find(name = 'div', attrs = {"class":"main_prodlist main_prodlist_list"})
    
#     # 각 상품 리스트의 영역 전체를 구함
#     eachProduct = totalProduct.findAll(name = "div", attrs = {"class":"prod_main_info"})
    
    # 상품명 구하기
    productName = bs.select('div > div.prod_info > p > a')
    productLen = len(productName)

#     itemNum = len(eachProduct)

    columnList = []
    

    for j in range(0, productLen):
#         columnList.append(eachProduct[j].find(name = "div", attrs = {"class":"prod_info"})\
#                           .find(name = 'p', attrs = {"class":"prod_name"}).find(name = 'a').text.strip())

        
        # a href 태그의 name 속성명의 속성값이 productName인 것을 이용
        driver.find_elements_by_name("productName")[j].click()
        time.sleep(sleepTime)
        
        # 탭 번호를 변수화
        firstTab = driver.window_handles[0]
        secondTab = driver.window_handles[1]

        driver.switch_to.window(secondTab)
        time.sleep(sleepTime)

        # 개별 상품 웹페이지 소스코드 변수에 저장
        eachBandHtml = driver.page_source

        # 태그 정보만 추출하여 변수에 저장
        bs = bs4.BeautifulSoup(eachBandHtml, 'html.parser')

        

        if bs.find('tr', {'class':'lowest'}) != None:

            # 최저가 정보 추출

            spanTag = bs.find('span', {'class':'lwst_prc'})

            columnList.append(spanTag.find('em', {'class':'prc_c'}).text.replace(',',""))

 

            # 최저가 구매사이트 추출

            siteTag = bs.find('tr', {'class':'lowest'})

            columnList.append(siteTag.find('a')['href'])

        else:

            nopriceTag = bs.find('div',{'class':'lowest_area lowest_blank'})

            columnList.append(nopriceTag.find('strong', {'class':'pnt'}).text)

            columnList.append('')

        

        # 테이블 정보를 포함한 상위태그 추출

        tableTag = bs.find('table', {'class':'spec_tbl'})

        tbodyTag = tableTag.find('tbody')

 

        tdTags = tbodyTag.findAll('td')

        thTags = tbodyTag.findAll('th', {'class':'tit'})

 

        for k in range(0, len(tdTags)):

            if thTags[k].text != '':

                if j == 0 and i == 0:

                    colHeaderList.append(thTags[k].text)

                    

                if k == 0:

                    columnList.append(tdTags[k].text.strip().split(' ')[0])

                else:    

                    columnList.append(tdTags[k].text)

 

        rowList.append(columnList)
        columnList = []

        
        driver.close()
        driver.switch_to.window(firstTab)

In [ ]:
# 참고 : 송호성 추천 방법.
# 해당 방법을 사용하면 태그를 찾아내는 부분은 태그를 못 찾을 경우 몇 차례 더 시도하도록 만든다.

searchFlag_2 = True
        notFoundCount = 0
        while (searchFlag_2):
            try:
                # 아이템 페이지에서 테이블 긁어오기
                data = driver.page_source
                bs = bs4.BeautifulSoup(data, 'html.parser')
                divTag = bs.find("div", attrs={"class":"malllowprice"})
                div2Tag = divTag.find("div", {"class":"malllow__cont"})
                tableTag = div2Tag.find('table', {'class':'malllow__list'})
                tbodyTag = tableTag.find('tbody', {'id':'vip_malllow_item_list_id'})
                trTag = tbodyTag.find('tr')
                tdTag = trTag.find('td', {'class':'malllow__item price'})
#                 오류가 발생하지 않는다면 False를 받아내고 while문을 탈출
                searchFlag = False
            except:
                notFoundCount += 1
                print("Not Found", notFoundCount)
                print("reSearch!!")
                time.sleep(1)
                if notFoundCount > 6:
                    break
                continue

In [12]:
# 데이터 가공

## 중복된 데이터 제거를 위해 길이가 82인 row의 7번째 인덱스 삭제

delList = []

rowListLen = len(rowList)

for i in range(0, rowListLen):

    if len(rowList[i]) == 82:

        delList.append(i)

delColNum = 7

for i in delList:

    del rowList[i][delColNum]


## 연주차컬럼 정제. 20XX년 09월 -> 20XX09

yearweekIndex = 4

for i in range(0, len(rowList)):

    rowList[i][yearweekIndex] = rowList[i][yearweekIndex].strip().replace('년 ','').replace('월','')
# 데이터 저장

# smartBand = pd.DataFrame(rowList, columns = colHeaderList)
smartBand = pd.DataFrame(rowList, columns = colHeaderList)

smartBand.to_csv('./smartBand.csv', index=False)

 

ValueError: 81 columns passed, passed data had 80 columns

In [2]:
# 데이터 저장

smartBand = pd.DataFrame(rowList, columns = colHeaderList)

smartBand.to_csv('./smartBand.csv', index=False)

In [8]:
import time

start_time = time.time()


#이곳에 코드를 넣습니다.


print("소요시간 : %s sec" % (time.time() - start_time))

    0
0   1
1   2
2   3
3   4
4   5
5   6
6   7
7   8
8   9
9  10
--- 0.0012850761413574219 seconds ---
